# CNN model

In [19]:
# Imports

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

# Local test setup
tf.config.threading.set_inter_op_parallelism_threads(6) # 8
tf.config.threading.set_intra_op_parallelism_threads(20) # 24

### 1. Data Preprocessing

In [ ]:
# Loading CIFAR-10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

print('Training labels shape:', y_train.shape)
print('Testing labels shape:', y_test.shape)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(labels)

In [ ]:
# Visualizing some images

plt.figure(figsize = (8, 8))

for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(labels[y_train[i][0]])

plt.show()

In [22]:
# One-hot encoding the labels

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
# Normalizing the Images

x_train = x_train / 255
x_test = x_test / 255

print(x_train[0].min())
print(x_train[0].max())

In [ ]:
# Resizing images

x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

print('Resized training image size:', x_train[0].shape)
print('Resized testing image size:', x_test[0].shape)

In [25]:
# Creating validation set

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)

# Augmenting data

augment = ImageDataGenerator(
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    # rotation_range = 10,      # Most images in the dataset have a natural, upright orientation.
    zoom_range = 0.1)

augment.fit(x_train)

### 2. Model Architecture

In [ ]:
# Defining the model

model = Sequential()

model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (64, 64, 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(256, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.3))

model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.3))

model.add(Dense(10, activation = 'softmax'))

# Model summary

model.summary()

### 3. Model Training

In [ ]:
# Compiling the model

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

# Early stopping / Learning rate scheduler

stopping = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)
scheduler = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 3, min_lr = 1e-6)

# Training the model (done in Colab)

model = load_model('models/cnn_model_0_88.keras')

# model.fit(augment.flow(x_train, y_train, batch_size = 256), epochs = 50, validation_data = (x_val, y_val), callbacks = [scheduler, stopping])

# Evaluating the model on the test data

test_loss, test_accuracy = model.evaluate(x_test, y_test)

print('Test data loss:', test_loss)
print('Test data accuracy:', test_accuracy)